In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install imagecodecs
!pip install segmentation-models-pytorch
!pip install optuna

In [ ]:
import numpy as np
import skimage.io
import torch
import torch.nn as nn
from torchvision.transforms import Compose, Resize, Grayscale
import imagecodecs
from pathlib import Path
import os
import optuna
import segmentation_models_pytorch as smp
from tensorflow import keras
import zipfile

In [ ]:
class ToFloat():
    def __call__(self, x):
        return x.astype(np.float32)

class ToTensor():
    def __call__(self, x):
        if len(x.shape) == 2:
            return torch.from_numpy(x.copy()).unsqueeze(0)
        elif len(x.shape) == 3:
            if x.shape[2]==3:
              return torch.permute(torch.from_numpy(x.copy()),(2,0,1))
            else:
              return torch.from_numpy(x.copy())
class RePermute():
    def __call__(self, x):
        if len(x.shape) == 3:
          if x.shape[0]==1:
            return torch.permute(x,(1,2,0))
        return x



class Normalize():
    def __init__(self, min=None, max =None, eps=1e-8):
        self.min = min
        self.max = max
        self.eps = eps

    def __call__(self, x):
        if self.min is not None and self.max is not None:
            return (x - self.min) / (self.max - self.min + self.eps)
        else:
            return (x - x.min()) / (x.max() - x.min() + self.eps)


class Instance2Semantic():
    """
    Converts instance segmentation masks into binary foreground/background masks
    """
    def __call__(self, x, threshold=0.5):
        return (x > threshold).astype(int)

# class to change cut image to square
class MakeSquare():
  def __call__(self, x):
      h, w = x.shape[:2]
      dh,dw =0,0
      m=h
      if h>w:
        dh=h-w
        m=w
      else:
        dw=w-h
        m=h
      a, b = np.random.randint(0, dh) if dh > 0 else 0, np.random.randint(0, dw) if dw > 0 else 0
      slices = (
            slice(a, a + m),
            slice(b, b + m)
      )
      x = x[slices]
      return x

def image_to_square(im,label,s = 480,transform = False):
  im = MakeSquare()(im)
  label=MakeSquare()(label)

  image_transforms = Compose([
    ToFloat(),
    ToTensor(),
    Grayscale(1),
    Resize((s,s)),
    RePermute(),
    Normalize(), # min=0, max=255
  ])
  label_transforms = Compose([
      Instance2Semantic(),
      ToTensor(),
      Resize((s,s)),
      RePermute()
  ])
  if transform:
    return image_transforms(im), label_transforms(label)

  return im, label

def create_dataset(
    dataset_dir, sequenc="images", labels="labels"):
    """
    Load images and masks from a dataset directory.
    """
    # create directory names
    img_dir = dataset_dir / sequenc
    msk_dir =  dataset_dir / labels
    # find out which frames have provided masks

    nf = sorted(os.listdir(img_dir))
    nf = [f"{img_dir}/{x}" for x in nf]
    frames =[]
    for f in nf:
      if ".tif" in f:
        frames.append(f)

    mask = [f"{msk_dir}/{x}" for x in sorted(os.listdir(msk_dir))]
    mask_list=[]
    for f in mask:
      if ".tif" in f:
        mask_list.append(f)


    # load images and masks
    images = [
        skimage.io.imread(fp)
        for fp in frames
    ]
    masks = [
        skimage.io.imread(fp)
        for fp in mask_list
    ]
    for i in range(len(images)):
      im,lab = image_to_square(images[i],masks[i],transform=True)
      images[i]=im.squeeze()
      masks[i]=lab.squeeze()

    return np.stack(images), np.stack(masks)

In [ ]:
path_to_training="drive/MyDrive/DeepLife/Training.zip"
path_to_test = "drive/MyDrive/DeepLife/Tuning_preprocessed.zip"
data_dir="data"
!mkdir data_dir

#"""
z = zipfile.ZipFile(path_to_test)
z.extractall(data_dir)
#"""
z = zipfile.ZipFile(path_to_training)
z.extractall(data_dir)
#"""

x_test, y_test = create_dataset(Path(f"{data_dir}/Tuning"), "images", "labels")
x_train, y_train = create_dataset(Path(f"{data_dir}/Training"), "images", "labels")
X_val = x_train[-20:]
X_train = x_train[:-20]
y_val = y_train[-20:]
y_train = y_train[:-20]

mkdir: cannot create directory ‘data_dir’: File exists


In [ ]:
# Define loss function and metrics outside the functions for reusability
loss_fn = torch.nn.BCEWithLogitsLoss()

def create_pspnet(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    dropout = trial.suggest_uniform('dropout', 0.0, 0.5)
    model = smp.PSPNet(
        encoder_name='resnet34',
        encoder_weights='imagenet',
        classes=1,
        activation=None, # Use None and apply sigmoid in loss or predictions
        dropout=dropout
    )
    # PyTorch models don't have a compile method like Keras
    # Define the optimizer here
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, optimizer

def objective(trial):
    global best_score
    # Get model and optimizer from create_pspnet
    model, optimizer = create_pspnet(trial)

    # Move model to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Convert data to PyTorch tensors and move to device
    # Assuming X_train, y_train, X_val, y_val are numpy arrays
    # Add channel dimension for grayscale
    X_train_tensor = torch.from_numpy(X_train).float().unsqueeze(1).to(device)
    y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1).to(device)
    X_val_tensor = torch.from_numpy(X_val).float().unsqueeze(1).to(device)
    y_val_tensor = torch.from_numpy(y_val).float().unsqueeze(1).to(device)

    # Duplicate the single channel to simulate a 3-channel RGB image for the encoder
    X_train_tensor = X_train_tensor.repeat(1, 3, 1, 1)
    X_val_tensor = X_val_tensor.repeat(1, 3, 1, 1)


    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    epochs = 10

    # Simple manual training loop
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(X_train_tensor), batch_size):
            inputs = X_train_tensor[i:i+batch_size]
            labels = y_train_tensor[i:i+batch_size]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation step
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for i in range(0, len(X_val_tensor), batch_size):
                inputs = X_val_tensor[i:i+batch_size]
                labels = y_val_tensor[i:i+batch_size]

                outputs = model(inputs)
                val_loss += loss_fn(outputs, labels).item() * inputs.size(0)

                # Calculate accuracy (example, adjust for segmentation)
                # For binary segmentation with BCEWithLogitsLoss and activation=None:
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                val_correct += (predicted == labels).sum().item()
                val_total += labels.numel() # total number of pixels

        avg_val_loss = val_loss / len(X_val_tensor)
        val_acc = val_correct / val_total
        print(f"Epoch {epoch+1}/{epochs}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

    # Optuna optimizes based on validation metric
    # Save model if it's the best so far based on validation accuracy
    if 'best_score' not in globals() or val_acc > best_score: # Check if best_score is defined
        best_score = val_acc
        # Save the state_dict of the model
        # Make sure BEST_MODEL_PATH is defined
        if 'BEST_MODEL_PATH' in globals():
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            print(f"New best model saved with validation accuracy: {val_acc:.4f}")
        else:
            print("BEST_MODEL_PATH is not defined. Cannot save the best model.")


    # Optuna objective is typically to minimize, so return negative accuracy
    return -val_acc

# Ensure CUDA is available if possible
if torch.cuda.is_available():
    print("Using CUDA")
else:
    print("Using CPU")

# Define best_score and BEST_MODEL_PATH before calling optimize
best_score = -float('inf')
BEST_MODEL_PATH = 'best_pspnet_model.pth' # Use .pth extension for PyTorch models

study = optuna.create_study(direction='minimize') # Use minimize since we return negative accuracy
study.optimize(objective, n_trials=20)

print("Best hyperparameters:", study.best_params)
print(f"Best model state_dict saved to {BEST_MODEL_PATH}")

[I 2025-05-15 18:40:36,757] A new study created in memory with name: no-name-068ebfc8-04e3-44c7-9baf-b6fdb3bfa999


Using CUDA


<ipython-input-6-943d1ab29eb5>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
<ipython-input-6-943d1ab29eb5>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.0, 0.5)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is re

Epoch 1/10, Val Loss: 0.5867, Val Accuracy: 0.9050
Epoch 2/10, Val Loss: 0.3242, Val Accuracy: 0.9170
Epoch 3/10, Val Loss: 0.2207, Val Accuracy: 0.9214
Epoch 4/10, Val Loss: 0.1938, Val Accuracy: 0.9242
Epoch 5/10, Val Loss: 0.1884, Val Accuracy: 0.9253
Epoch 6/10, Val Loss: 0.1809, Val Accuracy: 0.9308
Epoch 7/10, Val Loss: 0.1740, Val Accuracy: 0.9345
Epoch 8/10, Val Loss: 0.1674, Val Accuracy: 0.9370
Epoch 9/10, Val Loss: 0.1639, Val Accuracy: 0.9379


[I 2025-05-15 18:41:53,707] Trial 0 finished with value: -0.9364856770833333 and parameters: {'lr': 2.4819647128070413e-05, 'dropout': 0.23246865879648398, 'batch_size': 32}. Best is trial 0 with value: -0.9364856770833333.


...

Epoch 10/10, Val Loss: 0.5771, Val Accuracy: 0.6734
Epoch 1/10, Val Loss: 0.1704, Val Accuracy: 0.9449
Epoch 2/10, Val Loss: 0.1596, Val Accuracy: 0.9509
Epoch 3/10, Val Loss: 0.1238, Val Accuracy: 0.9543
Epoch 4/10, Val Loss: 0.1293, Val Accuracy: 0.9511
Epoch 5/10, Val Loss: 0.1512, Val Accuracy: 0.9473
Epoch 6/10, Val Loss: 0.2155, Val Accuracy: 0.9316
Epoch 7/10, Val Loss: 0.1355, Val Accuracy: 0.9477
Epoch 8/10, Val Loss: 0.1412, Val Accuracy: 0.9480
Epoch 9/10, Val Loss: 0.1535, Val Accuracy: 0.9467


[I 2025-05-15 19:05:18,962] Trial 18 finished with value: -0.9473891059027778 and parameters: {'lr': 0.0010577928019661078, 'dropout': 0.44504003848208457, 'batch_size': 32}. Best is trial 14 with value: -0.9530438368055556.


Epoch 10/10, Val Loss: 0.1275, Val Accuracy: 0.9474
Epoch 1/10, Val Loss: 2.0185, Val Accuracy: 0.6370
Epoch 2/10, Val Loss: 0.4279, Val Accuracy: 0.9441
Epoch 3/10, Val Loss: 0.3165, Val Accuracy: 0.9598
Epoch 4/10, Val Loss: 0.3084, Val Accuracy: 0.9177
Epoch 5/10, Val Loss: 0.1592, Val Accuracy: 0.9618
Epoch 6/10, Val Loss: 0.3057, Val Accuracy: 0.8932
Epoch 7/10, Val Loss: 0.4973, Val Accuracy: 0.9193
Epoch 8/10, Val Loss: 0.2847, Val Accuracy: 0.9265
Epoch 9/10, Val Loss: 0.4141, Val Accuracy: 0.8772


[I 2025-05-15 19:06:35,331] Trial 19 finished with value: -0.9460225694444444 and parameters: {'lr': 0.008193719916519128, 'dropout': 0.16065411690461734, 'batch_size': 16}. Best is trial 14 with value: -0.9530438368055556.


Epoch 10/10, Val Loss: 0.2275, Val Accuracy: 0.9460
Best hyperparameters: {'lr': 0.001321232630903497, 'dropout': 0.34615345880823367, 'batch_size': 16}
Best model state_dict saved to best_pspnet_model.pth


## Evaluation